# Import librairies

In [1]:
import geojson
import sys
import ee
import contextily

sys.path.append("..")
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.ops import unary_union
import warnings

warnings.filterwarnings("ignore")
import asyncio
import os.path
from os import path
import geopandas
from geopandas import GeoSeries

In [2]:
# Import file with all polygons from districts
file = geopandas.read_file("../material/filesgadm36_IND.gpkg")

In [13]:
# Authenticate to earth engine
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AX4XfWhnXcU8p1fEb_2WHlpBFF1ZjdI9PkYgu7UBnKbTbrUztwcLcO-1jcg



Successfully saved authorization token.


# Import data file

In [14]:
# Concatenate name of states and districts
file["total_name"] = file.apply(
    lambda x: f"{x['NAME_2']}, {x['NAME_1']}, India", axis=1
)
# Set all years we will take pictures of
years = np.arange(2000, 2019, 1)
years_kharif = [[f"{year}-04-16", f"{year}-10-15"] for year in years]
years_rabi = [[f"{year}-10-16", f"{year+1}-04-15"] for year in years]

In [15]:
names_to_search = file["total_name"].unique()

In [16]:
file.head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,GID_3,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry,total_name
0,IND,India,IND.1_1,Andaman and Nicobar,None,IND.1.1_1,Nicobar Islands,None,IND.1.1.1_1,n.a. ( 2304),None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((92.78778 9.24417, 92.78889 9.2...","Nicobar Islands, Andaman and Nicobar, India"
1,IND,India,IND.1_1,Andaman and Nicobar,None,IND.1.2_1,North and Middle Andaman,None,IND.1.2.1_1,n.a. ( 2178),None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((93.64841 14.93487, 93.64917 14...","North and Middle Andaman, Andaman and Nicobar,..."
2,IND,India,IND.1_1,Andaman and Nicobar,None,IND.1.3_1,South Andaman,None,IND.1.3.1_1,n.a. ( 2178),None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((93.83970 12.32082, 93.85775 12...","South Andaman, Andaman and Nicobar, India"
3,IND,India,IND.2_1,Andhra Pradesh,None,IND.2.1_1,Anantapur,None,IND.2.1.1_1,Anantapur,None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((77.35452 14.52155, 77.34958 14...","Anantapur, Andhra Pradesh, India"
4,IND,India,IND.2_1,Andhra Pradesh,None,IND.2.1_1,Anantapur,None,IND.2.1.2_1,Dharmavaram,None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((77.35341 14.27068, 77.35244 14...","Anantapur, Andhra Pradesh, India"


In [17]:
# Group all polygons by state/district
file_group = file.groupby(by=["NAME_1", "NAME_2"])["geometry"].apply(
    lambda x: unary_union(x)
)

# Run import images

In [19]:
def inverse_coords(poly):
    x, y = poly.exterior.coords.xy
    return Polygon(zip(y, x))


# Choose NDVI collection for image (for vegetation, cf. README)
l8 = ee.ImageCollection("MODIS/006/MOD13Q1").select("NDVI")
errors = []

nb_images = len(file_group.index) * len(years_kharif)
# compt = 0

# Define function for importing images
async def import_image(name, j):
    # compt+=1
    # print(f"Importing images is {(100*compt)//nb_images} % complete", end="\r")

    # Get the polygon for the state/district
    poly = file_group.loc[name[0], name[1]]
    d_kharif = years_kharif[j]
    d_rabi = years_rabi[j]

    # If there is already the image, just ignore
    if path.exists(f"../image_data/{d_kharif}_{name}.png"):
        return

    # If this is a multi polygon or single polygogn
    try:
        polys = list(poly)
    except:
        polys = poly

    # Set coordinates in the right format
    geopoly = GeoSeries(polys)
    try:
        geopoly = geopoly.apply(inverse_coords)
    except:
        pass
    # print(geopoly)
    geopoly = geopoly.set_crs("EPSG:4326")

    # Get bouding box
    west, south, east, north = (
        geopoly.bounds["miny"].min(),
        geopoly.bounds["minx"].min(),
        geopoly.bounds["maxy"].max(),
        geopoly.bounds["maxx"].max(),
    )

    # Get median image for both season and for the bouding box
    kharif = l8.filterDate(d_kharif[0], d_kharif[1]).mosaic()  # .filterBounds(houston)\
    rabi = l8.filterDate(d_rabi[0], d_rabi[1]).mosaic()  # .filterBounds(houston)\

    # Visualization parameters (mainly taken from examples in the earth engine API documentation)
    viz_params = {
        "bands": ["NDVI"],
        # "gain": '0.05',
        "scale": 1,
        "min": 0.0,
        "max": 9000.0,
        "palette": [
            "FFFFFF",
            "CE7E45",
            "DF923D",
            "F1B555",
            "FCD163",
            "99B718",
            "74A901",
            "66A000",
            "529400",
            "3E8601",
            "207401",
            "056201",
            "004C00",
            "023B01",
            "012E01",
            "011D01",
            "011301",
        ],
    }

    try:
        await asyncio.sleep(1)
        # Get url to download images
        url_kharif = kharif.getMapId(viz_params)["tile_fetcher"].url_format
        url_rabi = rabi.getMapId(viz_params)["tile_fetcher"].url_format

        # Get images with contextily library
        im_kharif, im_kharif_ext = contextily.bounds2img(
            west, south, east, north, ll=True, source=url_kharif
        )
        im_rabi, im_rabi_ext = contextily.bounds2img(
            west, south, east, north, ll=True, source=url_rabi
        )

        im_kharif = Image.fromarray(im_kharif)
        im_rabi = Image.fromarray(im_rabi)

        # Save images
        im_kharif.save(f"../image_data/{d_kharif}_{name}.png")
        im_rabi.save(f"../image_data/{d_rabi}_{name}.png")
    except:
        await asyncio.sleep(1)
        errors.append([name, j])


# Asynchronous function which run the script
async def main():
    for name in file_group.index:
        for j, _ in enumerate(years_kharif):
            await (import_image(name, j))


# coroutines = [import_image(name, j) for name in file_group.index for j,_ in enumerate(years_kharif)]
# await asyncio.gather(*coroutines)
import nest_asyncio

nest_asyncio.apply()
loop = asyncio.get_event_loop()
loop.run_until_complete(main())